In [36]:
#Importar las bibliotecas necesarias
# Crear una instancia de SparkSession

In [1]:
import findspark
findspark.init()

import pandas as pd
import pyspark

from pyspark.sql import functions as f
from pyspark.sql import SparkSession

from pyspark.sql.functions import lit

spark = SparkSession.builder.getOrCreate()


In [2]:
# 1. Crear un DataFrame con datos de personas, que incluya nombre, edad y ciudad de residencia.

In [3]:

data = [("Alice", 25, "New York"),
        ("Bob", 30, "Los Angeles"),
        ("Charlie", 22, "Chicago")]

df = spark.createDataFrame(data, ["Nombre", "Edad", "Ciudad"])
df.show()


+-------+----+-----------+
| Nombre|Edad|     Ciudad|
+-------+----+-----------+
|  Alice|  25|   New York|
|    Bob|  30|Los Angeles|
|Charlie|  22|    Chicago|
+-------+----+-----------+



In [ ]:
# 2. Mostrar solo los nombres de las personas del DataFrame.

In [4]:
df.select("Nombre").show()

+-------+
| Nombre|
+-------+
|  Alice|
|    Bob|
|Charlie|
+-------+



In [ ]:
# 3. Filtrar personas cuya edad sea mayor o igual a 25.

In [5]:
df.filter(df.Edad >= 25).show()

+------+----+-----------+
|Nombre|Edad|     Ciudad|
+------+----+-----------+
| Alice|  25|   New York|
|   Bob|  30|Los Angeles|
+------+----+-----------+



In [ ]:
#4. Agregar una nueva columna "Pais" con un valor constante para todas las filas.


In [6]:
df.withColumn("Pais", lit("EEUU")).show()


+-------+----+-----------+----+
| Nombre|Edad|     Ciudad|Pais|
+-------+----+-----------+----+
|  Alice|  25|   New York|EEUU|
|    Bob|  30|Los Angeles|EEUU|
|Charlie|  22|    Chicago|EEUU|
+-------+----+-----------+----+



In [ ]:
# 5. Calcular el promedio de edad de todas las personas.

In [7]:
df.agg(f.avg("Edad")).collect()[0][0]

25.666666666666668

In [ ]:
# 6. Ordenar el DataFrame por edad en orden descendente.

In [8]:
df.sort("Edad", ascending=False).show()

+-------+----+-----------+
| Nombre|Edad|     Ciudad|
+-------+----+-----------+
|    Bob|  30|Los Angeles|
|  Alice|  25|   New York|
|Charlie|  22|    Chicago|
+-------+----+-----------+



In [9]:
# 7. Agrupar por ciudad y calcular la cantidad de personas en cada ciudad.
df.groupBy("Ciudad").count().show()

+-----------+-----+
|     Ciudad|count|
+-----------+-----+
|   New York|    1|
|Los Angeles|    1|
|    Chicago|    1|
+-----------+-----+



In [10]:
# 8. Renombrar la columna "Nombre" a "NombreCompleto".
df.withColumnRenamed('Nombre', 'NombreCompleto').show()

+--------------+----+-----------+
|NombreCompleto|Edad|     Ciudad|
+--------------+----+-----------+
|         Alice|  25|   New York|
|           Bob|  30|Los Angeles|
|       Charlie|  22|    Chicago|
+--------------+----+-----------+



In [11]:
# 9. Eliminar la columna "Edad" del DataFrame.
df.drop('Edad').show()

+-------+-----------+
| Nombre|     Ciudad|
+-------+-----------+
|  Alice|   New York|
|    Bob|Los Angeles|
|Charlie|    Chicago|
+-------+-----------+



In [12]:
# 10. Realizar una consulta SQL en el DataFrame para seleccionar personas mayores de 20 años.
df.createOrReplaceTempView("temp_table")
spark.sql("select * from temp_table where Edad > 20").show()

+-------+----+-----------+
| Nombre|Edad|     Ciudad|
+-------+----+-----------+
|  Alice|  25|   New York|
|    Bob|  30|Los Angeles|
|Charlie|  22|    Chicago|
+-------+----+-----------+



In [13]:
# 11. Calcular la suma total de todas las edades.
spark.sql("select sum(Edad) as Total_Edad from temp_table").show()

+----------+
|Total_Edad|
+----------+
|        77|
+----------+



In [14]:

# 12. Calcular la edad mínima y máxima de todas las personas.
spark.sql("select max(Edad) as Max_Edad,min(Edad) as Min_Edad from temp_table").show()

+--------+--------+
|Max_Edad|Min_Edad|
+--------+--------+
|      30|      22|
+--------+--------+



In [15]:
# 13. Filtrar personas cuya ciudad de residencia sea "Chicago" y edad sea menor de 30.
spark.sql("select * from temp_table where Ciudad = 'Chicago' and Edad < 30").show()

+-------+----+-------+
| Nombre|Edad| Ciudad|
+-------+----+-------+
|Charlie|  22|Chicago|
+-------+----+-------+



In [16]:
# 14. Agregar una nueva columna "EdadDuplicada" que contenga el doble de la edad.
df.withColumn("EdadDuplicada", df.Edad * 2).show()

+-------+----+-----------+-------------+
| Nombre|Edad|     Ciudad|EdadDuplicada|
+-------+----+-----------+-------------+
|  Alice|  25|   New York|           50|
|    Bob|  30|Los Angeles|           60|
|Charlie|  22|    Chicago|           44|
+-------+----+-----------+-------------+



In [19]:
# 15. Convertir todas las edades en años a meses.
df.withColumn("Edad", df.Edad * 12).show()

+-------+----+-----------+
| Nombre|Edad|     Ciudad|
+-------+----+-----------+
|  Alice| 300|   New York|
|    Bob| 360|Los Angeles|
|Charlie| 264|    Chicago|
+-------+----+-----------+



In [23]:
# 16. Contar el número total de personas en el DataFrame.
spark.sql("select count(distinct(Nombre)) as Cantidad_de_Personas from temp_table").show()

+--------------------+
|Cantidad_de_Personas|
+--------------------+
|                   3|
+--------------------+



In [24]:
# 17. Filtrar personas cuya edad sea un número par.
df.filter(f.expr("Edad % 2 == 0")).show()

+-------+----+-----------+
| Nombre|Edad|     Ciudad|
+-------+----+-----------+
|    Bob|  30|Los Angeles|
|Charlie|  22|    Chicago|
+-------+----+-----------+



In [32]:
# 18. Calcular la cantidad de personas por rango de edades (0-20, 21-40, 41-60, 61+).
from pyspark.sql.functions import expr
cond = """case when Edad > 60 then '61+'
               else case when Edad >= 41 then '41-60'
                    else case when Edad >= 21 then '21-40'
                         else '0-20'
                              end
                         end
                end as cantidad"""

newdf = df.withColumn("Edad_cantidad", expr(cond))

newdf.groupBy("Edad_cantidad").count().show()


+-------------+-----+
|Edad_cantidad|count|
+-------------+-----+
|        21-40|    3|
+-------------+-----+



In [34]:
# 19. Contar cuántas personas tienen el mismo nombre.
df.groupBy("Nombre").count().show()

+-------+-----+
| Nombre|count|
+-------+-----+
|  Alice|    1|
|    Bob|    1|
|Charlie|    1|
+-------+-----+



In [35]:
# 20. Concatenar las columnas "Nombre" y "Ciudad" en una nueva columna llamada "InformacionPersonal".
df.withColumn("InformacionPersonal", f.concat("Nombre", f.lit(", "), "Ciudad")).show()

+-------+----+-----------+-------------------+
| Nombre|Edad|     Ciudad|InformacionPersonal|
+-------+----+-----------+-------------------+
|  Alice|  25|   New York|    Alice, New York|
|    Bob|  30|Los Angeles|   Bob, Los Angeles|
|Charlie|  22|    Chicago|   Charlie, Chicago|
+-------+----+-----------+-------------------+

